# Exhaustive search - Flowshop problem

This notebook contains a hands-on the flowshop problem. We will focus on implementing the exact methods to solve this problem. These methods guarantee to find the optimal solution for our problem. Note that some heuristics such as NEH are also being implemented in order to initialize the best makespan.

### Table of content
- [Brute force](#Brute-force)
- [Heuristics](#Heuristics)
- [Branch & Bound](#Branch-&-Bound)
- [Tests](#Tests)
   


### References
- [Finding an Optimal Sequence in the Flowshop Scheduling Using Johnson’s Algorithm](https://ijiset.com/vol2/v2s1/IJISET_V2_I1_50.pdf)
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)

In [25]:
import numpy as np
import matplotlib as plt
import itertools
import time
import math

# Brute force

In [2]:
def all_permutations(iterable):
    permutations = list(itertools.permutations(iterable))
    permutations_as_lists = [list(p) for p in permutations]
    return permutations_as_lists

In [3]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [4]:
def brute_force(processing_times, permutations):
    M = float('inf')
    sol = []
    for permutation in permutations:
        m = evaluate_sequence(permutation, processing_times)
        if m < M:
            M = m
            sol = permutation
    return sol, M

# Heuristics

## NEH Algorithm

In [5]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [6]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [7]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

## Johnson Algorithm

In [8]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        
    return m1+m2

### Tests

In [9]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(7,2), low=5, high=23)
print(rnd_data, "\n")

start_time = time.time()
sol = johnson_method(rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Johnson is {sol} with a makespan of {evaluate_sequence(sol, rnd_data)}')
print("Elapsed time:", elapsed_time, "seconds")

[[ 5 18]
 [13 11]
 [ 9 20]
 [15  6]
 [ 9 10]
 [18 17]
 [20  7]] 

Best sequence found by Johnson is [0, 2, 4, 5, 1, 6, 3] with a makespan of 95.0
Elapsed time: 0.0 seconds


In [10]:
init_jobs = 7
init_job_list = list(range(init_jobs))
print(rnd_data, "\n")

start_time = time.time()
sequence_list = all_permutations(init_job_list)
sol, M = brute_force(rnd_data, sequence_list)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Brute Force {sol} with a makespan of {M}')
print("Elapsed time:", elapsed_time, "seconds")

[[ 5 18]
 [13 11]
 [ 9 20]
 [15  6]
 [ 9 10]
 [18 17]
 [20  7]] 

Best sequence found by Brute Force [0, 1, 2, 4, 5, 6, 3] with a makespan of 95.0
Elapsed time: 0.4567866325378418 seconds


# Branch & Bound

- Branch & Bound.
- Branch & Bound pure.

In [11]:
# Define the Node structure of the seach tree that we will be using
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=1e100):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound
    def __str__(self):
        return f"Node(jobs={self.jobs}, remaining_jobs={self.remaining_jobs}, lower_bound={self.lower_bound})"

## Branch & Bound

In [17]:
def branch_and_bound(processing_times, initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            child_lower_bound = evaluate_sequence(child_jobs, processing_times)
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
            if child_lower_bound < best_solution_cost:
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

## Branch & Bound pure

In [12]:
def branch_and_bound_pure(processing_times,initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            else:
                # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
    return best_solution, best_solution_cost, i

# Tests

## Random instance

In [37]:
rnd_data = np.random.randint(size=(8,5), low=10, high=50)
print(rnd_data, "\n")

[[35 49 46 11 44]
 [39 47 49 45 11]
 [38 35 22 36 28]
 [38 14 17 27 13]
 [48 17 11 42 39]
 [10 48 36 27 41]
 [42 18 24 21 23]
 [23 30 44 12 32]] 



In [38]:
initial_solution = [i for i in range(8)]
initial_cost = evaluate_sequence(initial_solution, rnd_data)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound_pure(rnd_data, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound pure:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of Branch & Bound pure:
Best sequence is [7, 5, 4, 1, 0, 2, 6, 3] with a makespan of 384.0.
No. Nodes visited is 69282.
Elapsed time of 15.603382110595703 seconds.


In [39]:
initial_solution = [i for i in range(8)]
initial_cost = evaluate_sequence(initial_solution, rnd_data)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(rnd_data, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of Branch & Bound:
Best sequence is [7, 5, 4, 1, 0, 2, 6, 3] with a makespan of 384.0.
No. Nodes visited is 26817.
Elapsed time of 8.128053426742554 seconds.


In [43]:
initial_solution, initial_cost = neh_algorithm(rnd_data)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(rnd_data, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound (with NEH Initialization):')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of Branch & Bound (with NEH Initialization):
Best sequence is [5, 7, 4, 1, 0, 2, 6, 3] with a makespan of 384.0.
No. Nodes visited is 26444.
Elapsed time of 7.4751317501068115 seconds.


In [40]:
start_time = time.time()
best_solution, best_cost = brute_force(rnd_data, all_permutations(range(8)))
elapsed_time = time.time() - start_time

print(f'Results of Brute Force:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. of tested solutions {math.factorial(8)}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of Brute Force:
Best sequence is [5, 4, 7, 1, 0, 2, 6, 3] with a makespan of 384.0.
No. of tested solutions 40320.
Elapsed time of 6.222885847091675 seconds.


## Common Instance

In [41]:
instance_common = np.array([
    [71, 79, 85, 82, 83], 
    [84, 71, 66, 68, 81],
    [78, 81, 75, 72, 87],
    [78, 75, 66, 72, 88],
    [72, 88, 83, 85, 88],
    [86, 88, 79, 82, 78],
    [75, 66, 86, 78, 78],
    [80, 79, 66, 83, 78],
    [73, 73, 67, 77, 71],
    [80, 77, 83, 78, 67],
])

In [42]:
instance_common.shape

(10, 5)

In [45]:
initial_solution = [i for i in range(10)]
initial_cost = evaluate_sequence(initial_solution, instance_common)

start_time = time.time()
best_solution, best_cost, i = branch_and_bound(instance_common, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

Results of Branch & Bound:
Best sequence is [3, 2, 7, 6, 8, 0, 1, 4, 5, 9] with a makespan of 1102.0.
No. Nodes visited is 6235278.
Elapsed time of 1796.516449213028 seconds.
